```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de Datos no Estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Word2Vec - Modelos basados en secuencias

Los modelos basados en secuencias tienen la fortaleza que toman una secuencia de token (en un determinado orden) y generan una salida dependiendo del tipo de problema que se trate.
 - Seq2Class: Toman una secuencia de tokens y generan una clase
 - Seq2Seq: Toman una secuencia de token y generan otra secuencia de tokens. 
 
En este notebook exploraremos este tipo de modelos utilizandolos de la mano de word2vec:

## Prepración del ambiente

In [13]:
import warnings
warnings.filterwarnings('ignore')

Descarguemos algunos fragmentos de código para simplificar el trabajo

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/

--2020-09-03 19:34:32--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2832 (2.8K) [text/plain]
Saving to: ‘./Utils/TextNormalizer.py’

TextNormalizer.py   100%[===================>]   2.77K  --.-KB/s    in 0s      

2020-09-03 19:34:32 (22.6 MB/s) - ‘./Utils/TextNormalizer.py’ saved [2832/2832]

--2020-09-03 19:34:32--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

Descargamos el set de datos

In [2]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

--2020-09-03 19:34:43--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512573 (501K) [text/plain]
Saving to: ‘./Datasets/mascorpus/tweets_marketing.csv’

tweets_marketing.cs 100%[===================>] 500.56K  --.-KB/s    in 0.05s   

2020-09-03 19:34:43 (10.1 MB/s) - ‘./Datasets/mascorpus/tweets_marketing.csv’ saved [512573/512573]



Descargamos nuestros vectores de word2vec en español

In [10]:
!mkdir -p ./Models/Word2Vec
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin --directory-prefix ./Models/Word2Vec

--2020-09-03 19:39:42--  https://santiagxf.blob.core.windows.net/public/Word2Vec/model.bin
Resolving santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)... 52.239.220.32
Connecting to santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)|52.239.220.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1089974081 (1.0G) [application/octet-stream]
Saving to: ‘./Models/Word2Vec/model.bin’

model.bin           100%[===================>]   1.01G  18.1MB/s    in 64s     

2020-09-03 19:40:47 (16.2 MB/s) - ‘./Models/Word2Vec/model.bin’ saved [1089974081/1089974081]



Instalamos las librerias necesarias

In [3]:
!pip install unidecode
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 245kB 4.8MB/s 
     |████████████████████████████████| 16.2MB 1.3MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172934 sha256=489215548f6a2a42cf51ea4db684cda1184ea5a1c314b83afa28f3aeefaa4f58
  Stored in directory: /tmp/pip-ephem-wheel-cache-h231rce_/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Cargamos el set de datos

In [4]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

## Preprocesamiento de texto

Al igual que con Topic Modeling, nuestro primer paso es preprocesar el texto. Para focalizarnos en Word2Vec en este modulo, les preparé un modulo TweetTextNormalizer que hará todo el preprocesamiento por nosotros. Pueden explorar los parametros que recibe el constructor de esta clase para ver que opciones podemos configurar como Stemmer, Lemmatization, etc.

En lo particular, estamos creando un TweetTextNormalizer que:
 - Aplicará un tokenizer especifico para Twitter
 - Eliminará stop words
 - Aplicará lemmatization
 - Eliminará URLs
 - Eliminará acentos
 - Eliminará las mayusculas
 
Adicionalmente, el parametro text_to_sequence=True indica que la salida de este proceso no serán oraciones sino que tokens.

In [6]:
from Utils.TextNormalizer import TweetTextNormalizer

In [7]:
normalizer = TweetTextNormalizer(preserve_case=False,text_to_sequence=True)

Podemos probar como funciona:

## Vectorización de las palabras

En las actividades anteriores utilizamos siempre un TF-IDF vectorizer para generar los vectores. En esta oportunidad utilizaremos Word2Vec utilizando un modelo pre-entrenado para el idioma español. Adicionalmente, vemos que este vectorizer tiene el parametro sequence_to_idx en Verdadero. Esto significa que no queremos que como salida obtengamos los vectores de Word2Vec, sino que queremos "el indice" que se corresponde a la palabra en una matriz de indice-palabra/vectores. 

In [11]:
from Utils.Word2VecVectorizer import Word2VecVectorizer

In [12]:
w2v = Word2VecVectorizer(model_path='Models/Word2Vec/model.bin', sequence_to_idx=True)
embedding_weights = w2v.get_weights()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


El método get_weights() construye la matríz de indice-palabra/vector que luego será utilizado para encontrar los vectores correspondientes de cada palabra. Esta matriz tiene dimensiones m x n, donde m es la cantidad de palabras del vocabulario y n la dimensión de los vectores de word2vec. En este caso trabajamos con vectores de dimensionalidad 100.

## Modelos basados en sequencias

Vimos que cuando aplicamos técnicas de Topic Modeling, intentamos reducir la cantidad de dimensiones de nuestras representaciones de palabras para luego utilizar un clasificador para resolver la tarea en cuestión. Sin embargo, la suponsición básica de ese tipo de modelos es que un texto no es mas que una distribución de palabras (bag of words). Sin embargo, nosotros sabemos que un texto es una secuencia de palabras donde importa el orden. Para capturar este tipo de propiedades podemos utilizar modelos basados en secuencias.

### Construirmos un modelo basado en secuencias

Para construir nuestro modelo, utilizaremos TensorFlow. En particular utilizaremos la API de Keras que nos permite componer modelos de redes neuronales como una secuencia de pasos o capas que se conectan en una dirección. 

Utilizemos los siguientes tipos de capas:
 *  **Embedding:** Esta capa transforma vectores que representan indices dentro de una matriz en representaciones vectoriales densas. Básicamente en este caso nos resolverá la busqueda de las representaciones vectoriales para nuestras palabras.
 * **SpatialDropout1D:** Este tipo de capas ayudan a promover la independencia entre filtros (feature maps). Funciona en forma analoga a Dropout pero en lugar de desconectar elementos individuales, desconecta el filtro completo.
 * **LSTM:** Long Short-Term Memory layer - Hochreiter 1997
 * **Dense:** Una típica capa de una red neuronal completamente conectada (fully connected)

Algunos detalles para notar:
 * *loss='sparse_categorical_crossentropy'*, este problema de clasificación (crossentropy) de más de una clase (categorical). Sin embargo, nuestro output produce probabilidades de cada una de las clases posibles (7) en forma one-hot encoding. 
 * *metrics=['accuracy']*: Si bien nuestra metrica es accuracy, Keras hará un promedio ponderado del accuracy de cada clase. Este es el comportamiento por defecto.

In [14]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D

In [15]:
def build_model(sequence_len, vocab_size, emdedding_size, embedding_weights):
    model = Sequential([
        Embedding(vocab_size, emdedding_size,
                  weights=[embedding_weights],
                  trainable=False,
                  mask_zero=True),
        SpatialDropout1D(0.2),
        LSTM(emdedding_size),
        Dense(7, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Podemos inspeccionar el modelo:

Para continuar utilizando la misma metodología que veniamos utilizando antes, embeberemos este clasificador diseñado en TensorFlow dentro de un pipeline en Scikit-Learn. Esto lo podemos lograr utilizando KerasClassifier. Este tipo de elementos se los conoce como Wrappers. Este paso no es necesario siempre, en nuestro caso solo porque estamos trabajando con Scikit-Learn como nuestro orquestador:

In [20]:
estimator = keras.wrappers.scikit_learn.KerasClassifier(
    build_fn=build_model, 
    epochs=50,
    sequence_len=max_seq_len,
    vocab_size=w2v.vocab_size,
    emdedding_size=w2v.emdedding_size,
    embedding_weights=embedding_weights)

### Ajustando la longitud de las secuencias

Los modelos basados en secuencias pueden adaptarse a cualquier longitud de secuencia, sin embargo, los parametros de nuestras redes neuronales deberan ser fijos. Para esto definiermos una longitud máxima de la secuencia que vamos analizar. Para esto podemos utilizar un valor especifico o utilizar el valor máximo de tokens que hay en nuestro corpus.

La siguiente clase PadSequenceTransformer es un modulo que les preparé para simplificar este procesamiento. El mismo se encarga de ajustar cualquier secuencia para que tenga exactamente max_seq_len. Cuando la lingitud es mejor, se completan con ceros.

In [17]:
from Utils.PadSequenceTransformer import PadSequenceTransformer

In [18]:
max_seq_len = 100

In [19]:
seq2seq = PadSequenceTransformer(max_len=max_seq_len)

## Creando nuestro pipeline

Construimos nuestro pipeline como siempre combinando los diferentes pasos:

In [21]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', w2v),
                           ('padder', seq2seq),
                           ('estimator', estimator)])

Entrenamos nuestro modelo

In [22]:
model = pipeline.fit(X=X_train, y=y_train)

Epoch 1/50
79/79 [==============================] - 8s 99ms/step - loss: 1.3594 - accuracy: 0.5514
Epoch 2/50
79/79 [==============================] - 8s 101ms/step - loss: 0.7594 - accuracy: 0.7687
Epoch 3/50
79/79 [==============================] - 8s 102ms/step - loss: 0.6256 - accuracy: 0.7969
Epoch 4/50
79/79 [==============================] - 8s 101ms/step - loss: 0.5636 - accuracy: 0.8167
Epoch 5/50
79/79 [==============================] - 8s 96ms/step - loss: 0.5219 - accuracy: 0.8263
Epoch 6/50
79/79 [==============================] - 7s 89ms/step - loss: 0.4777 - accuracy: 0.8342
Epoch 7/50
79/79 [==============================] - 7s 92ms/step - loss: 0.4566 - accuracy: 0.8433
Epoch 8/50
79/79 [==============================] - 8s 99ms/step - loss: 0.4350 - accuracy: 0.8560
Epoch 9/50
79/79 [==============================] - 8s 98ms/step - loss: 0.4142 - accuracy: 0.8516
Epoch 10/50
79/79 [==============================] - 8s 98ms/step - loss: 0.4002 - accuracy: 0.8639
Epoch 

## Evalución de los resultados

Probamos su performance utilizando el test set

In [23]:
predictions = model.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.96      0.85      0.90       110
  AUTOMOCION       0.87      0.78      0.82       148
       BANCA       0.84      0.82      0.83       198
     BEBIDAS       0.88      0.89      0.88       223
    DEPORTES       0.76      0.85      0.80       216
      RETAIL       0.79      0.80      0.80       268
       TELCO       0.95      0.94      0.94        79

    accuracy                           0.84      1242
   macro avg       0.86      0.85      0.85      1242
weighted avg       0.84      0.84      0.84      1242

